In [1]:
import matplotlib.pyplot as plt
import numpy as np
from numpy.typing import NDArray
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, train_test_split
from sklearn.multioutput import MultiOutputRegressor
import pandas as pd
from scipy.optimize import minimize
import time

In [2]:
# 価格を生成する関数
def create_price(r_min: float, r_max: float, M: int) -> NDArray[np.float_]:
    # r_minとr_maxの間のランダムな0.1刻みの少数をM個生成
    price = np.random.uniform(r_min, r_max, M)
    price = np.round(price, 1)

    return price


# alphaを作成する関数
def alpha_star(M: int) -> NDArray[np.float_]:
    alpha_star = np.random.uniform(M, 3 * M, size=M)
    return alpha_star


# betaを作成する関数
def beta_star(M: int, M_prime: int) -> NDArray[np.float_]:
    beta_star = np.zeros((M, M_prime))

    for m in range(M):
        for m_prime in range(M_prime):
            if m == m_prime:
                beta_star[m, m_prime] = np.random.uniform(-2 * M, -M)
            else:
                beta_star[m, m_prime] = np.random.uniform(0, 1)

    return beta_star


def quantity_function(
    price: NDArray[np.float_],
    alpha: NDArray[np.float_],
    beta: NDArray[np.float_],
    delta: float = 0.1,  # ノイズレベルを指定（例として0.1を使用）
) -> list[float]:
    M = len(price)
    quantity_list = []
    q_m_no_noise = []

    # ステップ1: ノイズなしのq_mを計算
    for m in range(M):
        sum_beta = 0
        for m_prime in range(M):
            sum_beta += beta[m][m_prime] * price[m_prime]
        quantity = alpha[m] + sum_beta
        q_m_no_noise.append(quantity)

    # E[q_m^2]を計算
    E_q_m_squared = np.mean(np.array(q_m_no_noise) ** 2)

    # ステップ2: ノイズの標準偏差sigmaを計算
    sigma = delta * np.sqrt(E_q_m_squared)

    # ステップ3: ノイズを加えて最終的なq_mを計算
    for m in range(M):
        epsilon = np.random.normal(0, sigma)
        quantity = q_m_no_noise[m] + epsilon
        quantity_list.append(quantity)

    return quantity_list


def sales_function(
    price: NDArray[np.float_], alpha: NDArray[np.float_], beta: NDArray[np.float_]
) -> list[float]:
    M = len(price)
    sales_list = []

    for m in range(M):
        sum_beta = 0
        for m_prime in range(M):
            sum_beta += beta[m][m_prime] * price[m_prime]

        quantity = alpha[m] + sum_beta
        sales_list.append(quantity * price[m])

    return sales_list

In [3]:
# 目的関数を定義（最大化問題を最小化問題に変換）
def sales_objective_function(prices, alpha, beta, M):
    return -sum(
        prices[m] * (alpha[m] + sum(beta[m][m_prime] * prices[m_prime] for m_prime in range(M)))
        for m in range(M)
    )

In [4]:
def sales_optimize(
    M: int,
    alpha: np.ndarray,
    beta: np.ndarray,
    prices_list: list[float],
) -> tuple[float, np.ndarray]:
    # 初期値として与えられたprices_listを使用
    initial_prices = np.full(M, 0.6)
    # 各価格の範囲を設定（0.6から1.0）
    bounds = [(0.6, 1.0) for _ in range(M)]
    # 最適化を実行
    result = minimize(
        sales_objective_function,
        initial_prices,
        args=(alpha, beta, M),
        bounds=bounds,
        method="L-BFGS-B",
    )
    # 最適な価格と目的関数の値を取得
    optimal_prices = result.x
    optimal_value = -result.fun  # 符号を反転して元の最大化問題の値に戻す
    return optimal_value, optimal_prices

In [5]:
# 目的関数を定義
def predict_objective_function(
    prices: NDArray[np.float_], intercepts: [float], coefs: [NDArray[np.float_]], M: int
) -> float:
    # 各変数の内容をデバッグ出力
    # print("prices:", prices)
    # print("intercepts:", intercepts)
    # print("coefs:", coefs)
    # print("M:", M)

    return -sum(
        prices[m]
        * (intercepts[m] + sum(coefs[m][m_prime] * prices[m_prime] for m_prime in range(M)))
        for m in range(M)
    )

In [6]:
# 予測と最適化を行う関数
def lb_predict_optimize(
    lb: list[float], M: int, X: NDArray[np.float_], Y: NDArray[np.float_]
) -> tuple[float, NDArray[np.float_]]:
    lr = MultiOutputRegressor(LinearRegression())
    lr.fit(X, Y)
    # 係数と切片を取得
    coefs = [estimate.coef_ for estimate in lr.estimators_]
    intercepts = [estimate.intercept_ for estimate in lr.estimators_]

    # 初期値として与えられたprices_listを使用
    initial_prices = np.full(M, 0.6)
    # 各価格の範囲を設定（0.6から1.0）
    bounds = [(lb[m], 1.0) for m in range(M)]
    # 最適化を実行
    result = minimize(
        predict_objective_function,
        initial_prices,
        args=(intercepts, coefs, M),
        bounds=bounds,
        method="L-BFGS-B",
    )
    # 最適な価格と目的関数の値を取得
    optimal_prices = result.x
    optimal_value = -result.fun  # 符号を反転して元の最大化問題の値に戻す
    return -optimal_value

In [7]:
# CVを行う関数
def cross_validation_lb(
    lb: list[float],
    X: NDArray[np.float_],
    y: NDArray[np.float_],
    M: int,
    K: int,
    prices_list: list[float],
) -> float:
    kf = KFold(n_splits=K, shuffle=True, random_state=0)
    optimal_sales_list = []

    bounds = [(lb[m], 1.0) for m in range(M)]

    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # trainで学習
        lr_tilda = MultiOutputRegressor(LinearRegression())

        # 係数、切片を取得
        lr_tilda.fit(X_train, y_train)
        coefs = [estimate.coef_ for estimate in lr_tilda.estimators_]
        intercepts = [estimate.intercept_ for estimate in lr_tilda.estimators_]

        # 初期値として与えられたprices_listを使用
        initial_prices = np.full(M, 0.6)
        # 各価格の範囲を設定（0.6から1.0）

        # 最適化を実行
        result = minimize(
            predict_objective_function,
            initial_prices,
            args=(intercepts, coefs, M),
            bounds=bounds,
            method="L-BFGS-B",
        )
        # 最適な価格と目的関数の値を取得
        optimal_prices = result.x

        # testで学習
        lr_hat = MultiOutputRegressor(LinearRegression())
        lr_hat.fit(X_test, y_test)

        quantity_hat = lr_hat.predict([optimal_prices])
        sales_hat = np.sum(quantity_hat * optimal_prices)

        optimal_sales_list.append(sales_hat)

    return -np.mean(optimal_sales_list)

In [8]:
def esitimate_lb_nelder(lb, M, X, y, K, z, adaptive=True):
    # adaptive=Trueの場合、optionsにadaptive=Trueを指定
    bounds_nelder = minimize(
        cross_validation_lb,
        lb,
        args=(X, y, M, K, z),
        method="Nelder-Mead",
        bounds=[(0.6, 1.0) for _ in range(M)],
        options={"adaptive": adaptive},
    )
    return bounds_nelder.fun, bounds_nelder.x

In [9]:
# CVを行う関数
def cross_validation_pr(
    pr: list[float],
    X: NDArray[np.float_],
    y: NDArray[np.float_],
    M: int,
    K: int,
    prices_list: list[float],
) -> float:
    kf = KFold(n_splits=K, shuffle=True, random_state=0)
    optimal_sales_list = []

    bounds = [(0.6, 1.0) for _ in range(M)]

    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # trainで学習
        lr_tilda = MultiOutputRegressor(LinearRegression())

        # 係数、切片を取得
        lr_tilda.fit(X_train, y_train)
        coefs = [estimate.coef_ for estimate in lr_tilda.estimators_]
        intercepts = [estimate.intercept_ for estimate in lr_tilda.estimators_]

        # 初期値として与えられたprices_listを使用

        # 各価格の範囲を設定（0.6から1.0）

        # 最適化を実行
        result = minimize(
            predict_objective_function,
            pr,
            args=(intercepts, coefs, M),
            bounds=bounds,
            method="L-BFGS-B",
        )
        # 最適な価格と目的関数の値を取得
        optimal_prices = result.x

        # testで学習
        lr_hat = MultiOutputRegressor(LinearRegression())
        lr_hat.fit(X_test, y_test)

        quantity_hat = lr_hat.predict([optimal_prices])
        sales_hat = np.sum(quantity_hat * optimal_prices)

        optimal_sales_list.append(sales_hat)

    return -np.mean(optimal_sales_list)

In [10]:
# 予測と最適化を行う関数
def predict_optimize(
    M: int, X: NDArray[np.float_], Y: NDArray[np.float_], prices_list: list[float]
) -> tuple[float, NDArray[np.float_]]:
    lr = MultiOutputRegressor(LinearRegression())
    lr.fit(X, Y)
    # 係数と切片を取得
    coefs = [estimate.coef_ for estimate in lr.estimators_]
    intercepts = [estimate.intercept_ for estimate in lr.estimators_]

    # 初期値として与えられたprices_listを使用
    initial_prices = np.full(M, 0.6)
    # 各価格の範囲を設定（0.6から1.0）
    bounds = [(0.6, 1.0) for _ in range(M)]
    # 最適化を実行
    result = minimize(
        predict_objective_function,
        initial_prices,
        args=(intercepts, coefs, M),
        bounds=bounds,
        method="L-BFGS-B",
    )
    # 最適な価格と目的関数の値を取得
    optimal_prices = result.x
    optimal_value = -result.fun  # 符号を反転して元の最大化問題の値に戻す
    return optimal_value, optimal_prices

In [11]:
# CVを行う関数
def cross_validation_ub(
    ub: list[float],
    X: NDArray[np.float_],
    y: NDArray[np.float_],
    M: int,
    K: int,
    prices_list: list[float],
) -> float:
    kf = KFold(n_splits=K, shuffle=True, random_state=0)
    optimal_sales_list = []

    bounds = [(0.6, ub[m]) for m in range(M)]

    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # trainで学習
        lr_tilda = MultiOutputRegressor(LinearRegression())

        # 係数、切片を取得
        lr_tilda.fit(X_train, y_train)
        coefs = [estimate.coef_ for estimate in lr_tilda.estimators_]
        intercepts = [estimate.intercept_ for estimate in lr_tilda.estimators_]

        # 初期値として与えられたprices_listを使用
        initial_prices = np.full(M, 0.6)
        # 各価格の範囲を設定（0.6から1.0）

        # 最適化を実行
        result = minimize(
            predict_objective_function,
            initial_prices,
            args=(intercepts, coefs, M),
            bounds=bounds,
            method="L-BFGS-B",
        )
        # 最適な価格と目的関数の値を取得
        optimal_prices = result.x

        # testで学習
        lr_hat = MultiOutputRegressor(LinearRegression())
        lr_hat.fit(X_test, y_test)

        quantity_hat = lr_hat.predict([optimal_prices])
        sales_hat = np.sum(quantity_hat * optimal_prices)

        optimal_sales_list.append(sales_hat)

    return -np.mean(optimal_sales_list)

In [12]:
def esitimate_ub_nelder(ub, M, X, y, K, z, adaptive=True):
    # adaptive=Trueの場合、optionsにadaptive=Trueを指定
    bounds_nelder = minimize(
        cross_validation_lb,
        ub,
        args=(X, y, M, K, z),
        method="Nelder-Mead",
        bounds=[(0.6, 1.0) for _ in range(M)],
        options={"adaptive": adaptive},
    )
    return bounds_nelder.fun, bounds_nelder.x

In [13]:
M = 10
alpha = alpha_star(M)
beta = beta_star(M, M)
r_m = 0.6
r_M = 1.0
# 価格を入れるリスト
price_list = []
# 売上を入れるリスト
quantity_list = []

# 価格と対応する売上を500件生成
for i in range(500):
    price = create_price(r_m, r_M, M)
    price_list.append(price)
    quantity = quantity_function(price, alpha, beta)
    quantity_list.append(quantity)

X = np.array(price_list).astype(float)
y = np.array(quantity_list).astype(float)

z = [0.6, 0.7, 0.8, 0.9, 1.0]

lb = [0.6 for _ in range(M)]

ub = [1.0 for _ in range(M)]

In [14]:
esitimate_lb_nelder_val_10, esitimate_lb_nelder_x_10 = esitimate_lb_nelder(lb, M, X, y, 5, z)

In [15]:
esitimate_lb_nelder_x_10

array([0.60776664, 0.60061517, 0.6       , 0.60941936, 0.60226671,
       0.60297635, 0.60058429, 0.6001315 , 0.60886766, 0.6045044 ])

In [16]:
esitimate_lb_nelder_val_10

-121.1733935826351

In [17]:
esitimate_ub_nelder_val_10, esitimate_ub_nelder_x_10 = esitimate_ub_nelder(ub, M, X, y, 5, z)

In [18]:
esitimate_ub_nelder_val_10

-121.30681120754582

In [19]:
esitimate_ub_nelder_x_10

array([0.93308548, 0.93743848, 0.60131709, 0.94609848, 0.94544615,
       0.75760996, 0.93074529, 0.95164446, 0.9663256 , 1.        ])

In [20]:
# M =20
M = 20
alpha = alpha_star(M)
beta = beta_star(M, M)
r_m = 0.6
r_M = 1.0
# 価格を入れるリスト
price_list = []
# 売上を入れるリスト
quantity_list = []

# 価格と対応する売上を500件生成
for i in range(500):
    price = create_price(r_m, r_M, M)
    price_list.append(price)
    quantity = quantity_function(price, alpha, beta)
    quantity_list.append(quantity)


X = np.array(price_list).astype(float)
y = np.array(quantity_list).astype(float)


z = [0.6, 0.7, 0.8, 0.9, 1.0]

lb = [0.6 for _ in range(M)]

ub = [1.0 for _ in range(M)]

esitimate_lb_nelder_val_20, esitimate_lb_nelder_x_20 = esitimate_lb_nelder(lb, M, X, y, 5, z)

esitimate_ub_nelder_val_20, esitimate_ub_nelder_x_20 = esitimate_ub_nelder(ub, M, X, y, 5, z)


In [21]:
esitimate_lb_nelder_x_20

array([0.60896163, 0.61196227, 0.61398809, 0.64799303, 0.60666934,
       0.6236495 , 0.60775417, 0.63622405, 0.60893563, 0.6       ,
       0.62231398, 0.64478541, 0.63248749, 0.73296149, 0.69880441,
       0.63817199, 0.61947751, 0.62997328, 0.75344931, 0.64440915])

In [22]:
esitimate_lb_nelder_val_20

-446.98734163837946

In [23]:
esitimate_ub_nelder_x_20

array([0.95839004, 0.81079632, 0.97392382, 0.81306904, 0.79115758,
       0.87999059, 0.92513665, 1.        , 0.87826464, 0.60018362,
       0.94971878, 0.76832858, 0.83605703, 0.73519944, 0.70031315,
       0.8618365 , 0.90933614, 0.89334665, 0.75580533, 0.93943498])

In [24]:
esitimate_ub_nelder_val_20

-447.4968397878467

In [25]:
# M = 30
M = 30

alpha = alpha_star(M)
beta = beta_star(M, M)
r_m = 0.6
r_M = 1.0
# 価格を入れるリスト
price_list = []
# 売上を入れるリスト
quantity_list = []

# 価格と対応する売上を500件生成
for i in range(500):
    price = create_price(r_m, r_M, M)
    price_list.append(price)
    quantity = quantity_function(price, alpha, beta)
    quantity_list.append(quantity)


X = np.array(price_list).astype(float)
y = np.array(quantity_list).astype(float)


z = [0.6, 0.7, 0.8, 0.9, 1.0]

lb = [0.6 for _ in range(M)]

ub = [1.0 for _ in range(M)]

estimate_lb_nelder_val_30, esitimate_lb_nelder_x_30 = esitimate_lb_nelder(lb, M, X, y, 5, z)

esitimate_ub_nelder_val_30, esitimate_ub_nelder_x_30 = esitimate_ub_nelder(ub, M, X, y, 5, z)


In [26]:
esitimate_lb_nelder_x_30

array([0.610923  , 0.63626872, 0.61724643, 0.63045103, 0.66199352,
       0.62128998, 0.6000244 , 0.73549225, 0.69719678, 0.62662285,
       0.6333611 , 0.6280546 , 0.60220315, 0.63601679, 0.64350982,
       0.62048473, 0.61321079, 0.65260868, 0.6       , 0.61974112,
       0.60096907, 0.61857691, 0.63900324, 0.76623627, 0.62459819,
       0.60143062, 0.60571084, 0.64329794, 0.60760994, 0.62112242])

In [27]:
estimate_lb_nelder_val_30

-798.4489635140787

In [28]:
esitimate_ub_nelder_x_30

array([0.96270773, 0.86532021, 0.62492819, 1.        , 0.65948545,
       0.87642377, 0.85473465, 0.73869897, 0.79138568, 0.6140682 ,
       0.89228063, 0.62897215, 0.96995702, 0.92247423, 0.96124896,
       0.9349    , 0.8272326 , 0.94781336, 0.60953453, 0.78107835,
       0.76606436, 0.82891984, 0.98107102, 0.77795619, 0.8133279 ,
       0.9523592 , 0.84788128, 0.82183001, 0.62511801, 0.69406324])

In [29]:
esitimate_ub_nelder_x_30

array([0.96270773, 0.86532021, 0.62492819, 1.        , 0.65948545,
       0.87642377, 0.85473465, 0.73869897, 0.79138568, 0.6140682 ,
       0.89228063, 0.62897215, 0.96995702, 0.92247423, 0.96124896,
       0.9349    , 0.8272326 , 0.94781336, 0.60953453, 0.78107835,
       0.76606436, 0.82891984, 0.98107102, 0.77795619, 0.8133279 ,
       0.9523592 , 0.84788128, 0.82183001, 0.62511801, 0.69406324])

In [30]:
# M = 40
M = 40

alpha = alpha_star(M)
beta = beta_star(M, M)

r_m = 0.6
r_M = 1.0
# 価格を入れるリスト
price_list = []
# 売上を入れるリスト
quantity_list = []

# 価格と対応する売上を500件生成
for i in range(500):
    price = create_price(r_m, r_M, M)
    price_list.append(price)
    quantity = quantity_function(price, alpha, beta)
    quantity_list.append(quantity)


X = np.array(price_list).astype(float)
y = np.array(quantity_list).astype(float)


z = [0.6, 0.7, 0.8, 0.9, 1.0]

lb = [0.6 for _ in range(M)]

ub = [1.0 for _ in range(M)]

esitimate_lb_nelder_val_40, esitimate_lb_nelder_x_40 = esitimate_lb_nelder(lb, M, X, y, 5, z)

esitimate_ub_nelder_val_40, esitimate_ub_nelder_x_40 = esitimate_ub_nelder(ub, M, X, y, 5, z)


In [31]:
esitimate_lb_nelder_x_40

array([0.62361964, 0.64502439, 0.6       , 0.61975522, 0.61895017,
       0.65946362, 0.6105385 , 0.63572359, 0.6597171 , 0.60954285,
       0.63592612, 0.66740874, 0.64184827, 0.65675147, 0.63647906,
       0.76771389, 0.68192217, 0.62844324, 0.63238027, 0.60876389,
       0.64524133, 0.61341658, 0.61405147, 0.62211412, 0.60400292,
       0.62661732, 0.62304703, 0.64788528, 0.63010476, 0.67185597,
       0.60701849, 0.64933208, 0.82271383, 0.64159372, 0.64917942,
       0.640565  , 0.61526416, 0.64897672, 0.77746969, 0.65598908])

In [32]:
esitimate_lb_nelder_val_40

-1768.3584689256338

In [33]:
esitimate_ub_nelder_x_40

array([0.99723759, 0.99951026, 0.60012389, 0.98561306, 0.99788819,
       0.99459201, 0.92517889, 1.        , 0.91065085, 0.80149636,
       0.95207271, 0.94134437, 0.96669323, 0.9106261 , 0.90537754,
       0.7710141 , 0.99579313, 0.91290046, 0.92108857, 0.95221319,
       0.95946389, 0.80380796, 0.9289922 , 0.99954695, 0.60034098,
       0.94597529, 0.99360621, 0.90106583, 0.99150287, 0.91578151,
       0.85800925, 0.67734772, 0.82702871, 0.94178755, 0.98897896,
       0.94541362, 0.84432737, 0.9579824 , 0.79010964, 0.91979097])

In [34]:
esitimate_ub_nelder_val_40

-1788.2407029293931

In [35]:
# M = 50
M = 50

alpha = alpha_star(M)
beta = beta_star(M, M)
r_m = 0.6
r_M = 1.0

# 価格を入れるリスト
price_list = []
# 売上を入れるリスト
quantity_list = []

# 価格と対応する売上を500件生成
for i in range(500):
    price = create_price(r_m, r_M, M)
    price_list.append(price)
    quantity = quantity_function(price, alpha, beta)
    quantity_list.append(quantity)


X = np.array(price_list).astype(float)
y = np.array(quantity_list).astype(float)


z = [0.6, 0.7, 0.8, 0.9, 1.0]

lb = [0.6 for _ in range(M)]

ub = [1.0 for _ in range(M)]

esitimate_lb_nelder_val_50, esitimate_lb_nelder_x_50 = esitimate_lb_nelder(lb, M, X, y, 5, z)


esitimate_ub_nelder_val_50, esitimate_ub_nelder_x_50 = esitimate_ub_nelder(ub, M, X, y, 5, z)


In [36]:
esitimate_lb_nelder_x_50

array([0.62004832, 0.65675416, 0.60767159, 0.61408478, 0.65318036,
       0.6410807 , 0.64599938, 0.60284301, 0.61238089, 0.64850064,
       0.62743836, 0.63294921, 0.60935488, 0.60660821, 0.63914821,
       0.60877849, 0.66158937, 0.60609799, 0.61214439, 0.60657103,
       0.64542431, 0.60835843, 0.60482525, 0.66118864, 0.60768749,
       0.62186441, 0.60875405, 0.64242582, 0.65409269, 0.64727782,
       0.60655999, 0.6       , 0.60848972, 0.62933901, 0.64461824,
       0.61506853, 0.62473601, 0.61596989, 0.66486192, 0.64909475,
       0.61050668, 0.60930874, 0.83241293, 0.63860001, 0.65159137,
       0.65779262, 0.66113795, 0.61075365, 0.66155869, 0.91919697])

In [37]:
esitimate_lb_nelder_val_50

-2787.8618422217883

In [38]:
esitimate_ub_nelder_x_50

array([0.99938798, 0.94425781, 0.99839963, 0.93586714, 0.98358051,
       0.86672632, 0.99681832, 0.99644828, 0.99425831, 0.99966562,
       0.92461303, 0.99901595, 0.85068263, 0.81769289, 0.96110865,
       0.98256336, 0.71760802, 0.99949232, 0.98389619, 0.99994932,
       1.        , 0.99983304, 0.99533005, 0.92999535, 0.81358691,
       0.99931215, 0.99991062, 0.99958378, 0.98082576, 0.66056449,
       0.97352604, 0.60343246, 0.99933446, 0.94526234, 0.97806466,
       0.99996154, 0.99974094, 0.99992281, 0.88003952, 0.99964019,
       0.99415915, 0.99401338, 0.8455081 , 0.99884012, 0.97077446,
       0.9859544 , 0.88769528, 0.99191694, 0.9298634 , 0.91731018])

In [39]:
esitimate_ub_nelder_val_50

-2834.38504412673